In [15]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import LineTokenizer
from nltk.tokenize import wordpunct_tokenize
import math
import torch
import nltk
nltk.download('punkt')

if torch.cuda.is_available():  
  dev = "cuda"
else:  
  dev = "cpu" 
device = torch.device(dev)
 
mname = 'Helsinki-NLP/opus-mt-ru-en'
mname = 'gsarti/opus-mt-tc-en-pl'
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
model.to(device)

lt = LineTokenizer()
batch_size = 8

#text_short = "Na tapczanie siedzi leń, nic nie robi cały dzień. O, wypraszam to sobie, jak to ja nic nie robię. A kto siedzi na tapczanie, a kto zjadł pierwsze śniadanie, a kto dziś pluł i łapał, a kto się wygłowę podrapał, a kto dziś zgubił kalosze. O, oproszę na tapczanie siedzi leń, nic nie robi cały dzień. Przepraszam, a tranu nie piłem, a uszu dzisiaj nie myłem, a nie urwałem guzika, a nie pokazałem języka, a nie chodziłem się z trzuc, to wszystko nazywa się nic na tapczanie siedzi leń. Nic nie robi cały dzień. Nie poszedł do szkoły, bo mu się nie chciało. Nie odrobił lekcji, bo czasu miał za mało. Nie zasznurował trzewików, bo nie miał ochoty. Nie powiedział dzień dobry, bo poza tym za dużo roboty. Nie napoił azorka, bo za daleko jest woda. Nie nakarmił kanarka, bo z czasu mu było szkoda. Miał zjeść kolacy, tylko usta mim lasną. Miał położyć się spać za sną. Sniło mu się, że nad czymś ogromnie się trudził. Tak się zmęczył tym snem, że się obudził."
#text_short = " ИНТРИГУЮЩАЯ МУЗЫКА Итоги года подводят и Европа, которая, если ещё и не потрескивает, то пошвам уже прищить. Проэкзит, правый поворот, раскалывающие всех мигранты, европейский лидер Германия в политическом параллете. В некоторых странах, между прочим, уже изымают детей из семей, если эти семьи отказываются от пропаганды гомосексуализма у себя. Сейчас у Невросии. Сейчас Польша называет себя главной жертвой Второй мировой. Но не аналия была её хичным инициатором. ДИНАМИЧНАЯ МУЗЫКА Если ультиматум нам предъявляют, то лонкомандующие направляют и балтийский флот в берегам, так сказать, Британии. Если ещё что-то скажут, Хрущёв сказал об нашу британцам, не достаточно две ракеты и ваших островов не будет. Закнулись вперёд на 20 лет. ДИНАМИЧНАЯ МУЗЫКА Больше, так сказать, прочеченной, чем Лондон, по отмыванию разных денег, вообще придумать сложно. Это раз. Эти крылатые слова, которым уже ни одна сотня лет, англичанка гадит. Стой уж стойчивы в русском языке, что, кажется, нестребимы. В 19 веке в России даже стало расхожим выражение англичанка гадит. И обозначало оно разные явные или нет, но всегда идущие во вред России действия англо-саксов. Именно Англия помогла уничтожить не только Рюриковичей, но и следующую правящую династию Романовых. У англо-саксов другая история. Это соронча, которая, приходя на новое место, должна его полностью зачистить. ДИНАМИЧНАЯ МУЗЫКА В Великобритании раскрыты новые подробности скандала с чиновниками-педофилами. Во властных кругах страны действовал целый подпольный клуб из вращенцев, который пользовался по кровительством разведки из рассекреченных документов. В администрации Тетчер знали о педофильском заговоре, но скрывали, чтобы не дискредитировать власть. Процветает массовая педофилия. В общей сложности унижения мы насилию подверглись около 1400 детей. Многие из этих случаев власти прекрасно знали. И, скорее всего, давно присекли бы издевательство быть жертвами коренными жителями страны. Согласно документам, нередко родители, пытавшиеся спасти своих детей, они были на летние, обвинены в том, что вступили в сексуальную связь по собственному желанию. Ответ оказался прост. Городские власти, полиция и все, кто сталкивался с этой проблемой, боялись прослыть недостаточно толерантными. И списывали все на разность культур. НАПРЯЖЕННАЯ МУЗЫКА На этой неделе Москву посетила антирочистская риторика и гейпропаганда в одном лице. Лаврову предстояло самое тяжелое в его работе. Сказать уважаемыми. Не добавив уважаемый дебилб***. Британский Лев уже не рычит, опищит. Британский Лев выглядит настолько убого с этими заявлениями. Слушайте, Тони Блейер уже после этих британгов извинялся. Пока, пока эти люди, которых представляют госпожа, мы извинялись только перед самими собой. Потому что это деньги налогоплатильщиков, потому что это жизни британских поданных. Они пока еще не извинились ни перед одной стороной, где была нога их солдат, где мирные жители гибли, где за счет их интриг и их шулирства руины превращалась в нормальную жизнь. НАПРЯЖЕННАЯ МУЗЫКА Заодно предложил у Лондона рассказать, что же все-таки случилось с Литвиненко, Березовским, перепелищным. Он точно умер на британской земле и чьи смерти? Так и не были внятно расследованными. При этом именно Россию британские сме обвиняют в развязывании так называемой холодной информационной войны, как это здесь называют. Называть белое-черном похоже для Британии становится привычным. И эта глава нынешнего британского правительства Терезе Мэй предпочла произнести свою самую агрессивную речь в адрес России, щедро приправив ее белизной своей усталой плоти. Как видим, глоток столь жаден, что оказалось не дорог. Бокал был взят не за ножку, как это всегда подобало в приличном обществе, а по-простому, как стакан. Это не муслима. Когда у Истан Черчин в 1956 году, задуматься, да, получил Нобелевскую премию по литературе, за книжку своих мемуаров, очень тенденциозных, не в режнонаписанных, он диктовал эти мемуары, получил Нобелевскую премию, и никто этому не удивился, уже тогда они в Англии и не в мире. Вот тогда стало понятно, что с Европой что-то не так."
text_short = "THE end of the year is also the end of Europe, which, if it doesn't crack, we're gonna stitch it up. Proexit, right-hand turn, cracking all migrants, European leader of Germany in a political parallel. In some countries, by the way, children are already taken away from their families if they refuse to promote homosexuality at home. Nevrosia now. Poland now calls itself the main victim of World War II. But it wasn't the analium that was her genius. A DYNAMIC MUSIC, if an ultimatum is presented to us, the commando also sends the Baltic fleet to the shores of Britain, so to speak. If anything else is said, Chrushchev told us about our British, there won't be enough missiles and your islands will be gone. Shut up for 20 years. A DYNAMIC MUSIC is more, so to speak, scrubbed than London on money laundering at all. These winged words, which aren't a hundred years old, are English women. Stand firm in Russian, which seems to be inexorable. In the 19th century, in Russia, even the expression of an English woman fucked up. And it meant different obvious or not, but always harmful to Russia, the actions of the Anglo-Saxes. It was England that helped destroy not only the Rürikovich but also the next ruling Romanov dynasty. The Anglo-Saxes have a different story. It's a soronche that comes to a new place and has to clean it up completely. A DYNAMIC MUSIC in the UK revealed new details of the paedophile officials' scandal. There was an entire underground spinal club in the country's power circles, which used blood-stained intelligence from declassified documents. At the Tetcher administration, they knew about the pedophile plot, but they hid it to avoid discrediting power. Mass pedophilia is flourishing. In total, we have been subjected to violence against some 1,400 children. Many of these cases were well known by the authorities. And it would probably have been a long time ago for the country's indigenous victims to be bullied. According to the documents, parents who were often trying to save their children were accused of having had sexual intercourse of their own free will during the summer. The answer turned out to be simple: the city authorities, the police, and all those who faced the problem were afraid of being insufficiently tolerant. And write it down to different cultures. This week Moscow visited anti-robber rhetoric and gay propaganda in one person. Lavrov was the hardest part of his job. To tell the respectable. Not adding a respectable dumbass. The British lion isn't crying anymore, he's gonna drink. The British Lion looks so miserable with these statements. Look, Tony Bleier apologised after those razors. So far, while these people are represented by Madame, we've only apologized to ourselves. Because it's the taxpayers' money, because it's the lives of the British plaintiffs. They have not yet apologized to any side where their soldiers' feet were, where civilians were killed, where their intrigue and their hooliganism turned the ruins into normal life. SOMEONE AT London offered to tell us what happened to Litvinenko, Berezovsky, the quail. He definitely died on British soil, and whose death? They were never properly investigated. But it is Russia that the British laugh is accused of unleashing the so-called cold information war, as it is called here. Calling white and black sounds like Britain's getting used to it. And this current head of the British government, Teresa May, chose to give his most aggressive speech against Russia, generously adding it to the whiteness of his tired flesh. As we can see, the sip is so thirsty, it wasn't expensive. The glass was not taken by the foot, as was always the case in a decent society, but simply as a glass. It's not a muslim. When Istan Churchin had, in 1956, thought, yes, he got the Nobel Prize in literature, for a book of his memoirs, very tendentious, unrewritten, he dictated these memoirs, he got the Nobel Prize, and no one was surprised by it, and they were already in England and not in the world. That's when it became clear that there was something wrong with Europe."

text_long = text_short

paragraphs = lt.tokenize(text_long)   
translated_paragraphs = []

for paragraph in paragraphs:
    sentences = sent_tokenize(paragraph, language="english")
    batches = math.ceil(len(sentences) / batch_size)     
    translated = []
    for i in range(batches):
        sent_batch = sentences[i*batch_size:(i+1)*batch_size]
        model_inputs = tokenizer(sent_batch, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
        with torch.no_grad():
            translated_batch = model.generate(**model_inputs)
        translated += translated_batch
    translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    translated_paragraphs += [" ".join(translated)]

translated_text = "\n".join(translated_paragraphs)

[nltk_data] Downloading package punkt to /home/fryderyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
translated_text

'Koniec roku to również koniec Europy, która, jeśli nie pęknie, zszyjemy ją. Proexit, skręt w prawo, złamanie wszystkich migrantów, europejskiego lidera Niemiec w równoległym kontekście politycznym. W niektórych krajach dzieci są już zabierane z rodzin, jeśli nie chcą propagować homoseksualizmu w domu. Teraz Nevrosia. Polska nazywa się obecnie główną ofiarą II wojny światowej. Ale to nie analium było jej geniuszem. MUZYKA DYNAMICZNA, jeśli nam zostanie przedstawiona ultimatum, komandos wysyła również flotę Bałtyku na brzegi Wielkiej Brytanii, że tak powiem. Jeśli coś jeszcze zostanie powiedziane, Chruszczow powiedział nam o naszych Brytyjczykach, nie będzie wystarczającej ilości pocisków, a wasze wyspy znikną. Zamknij się na 20 lat. MUZYKA DYNAMICZNA jest bardziej, że tak powiem, oczyszczona niż Londyn w ogóle o praniu pieniędzy. Te skrzydlate słowa, które nie mają stu lat, to angielskie kobiety. Stań mocno po rosyjsku, co wydaje się nieubłagane. W XIX wieku, w Rosji, nawet wyraz angie